# Домашнее задание 3
**Продвинутое машинное обучение**

In [83]:
import nltk
import numpy as np
import random

from collections import Counter
from nltk.util import ngrams
from random import shuffle

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Loading and preparing data

In [2]:
WAR_AND_PEACE_PATH = '/content/drive/MyDrive/adv-ml/WarAndPeace.txt'
WAR_AND_PEACE_ENG_PATH = '/content/drive/MyDrive/adv-ml/WarAndPeaceEng.txt'
ANNA_KARENINA_PATH = '/content/drive/MyDrive/adv-ml/AnnaKarenina.txt'

In [18]:
with open(WAR_AND_PEACE_PATH, 'r') as file:
    war_and_peace = file.read().replace('\n', ' ')

with open(WAR_AND_PEACE_ENG_PATH, 'r') as file:
    war_and_peace_eng = file.read().replace('\n', ' ')

with open(ANNA_KARENINA_PATH, 'r') as file:
    anna_karenina = file.read().replace('\n', ' ')

In [35]:
text_ru = ' '.join([war_and_peace, anna_karenina])

In [61]:
example_texts = ['В то время как у Ростовых танцевали в зале шестой англез под звуки от усталости фальшививших музыкантов и усталые официанты и повара готовили ужин, с графом Безуховым сделался шестой уже удар. Доктора объявили, что надежды к выздоровлению нет; больному дана была глухая исповедь и причастие; делали приготовления для соборования, и в доме была суетня и тревога ожидания, обыкновенные в такие минуты. Вне дома, за воротами, толпились, скрываясь от подъезжающих экипажей, гробовщики, ожидая богатого заказа на похороны графа.',
                 'Пьер, решившись во всем повиноваться своей руководительнице, направился к диванчику, который она ему указала. Как только Анна Михайловна скрылась, он заметил, что взгляды всех бывших в комнате больше чем с любопытством и с участием устремились на него. Он заметил, что все перешептывались, указывая на него глазами, как будто со страхом и даже с подобострастием.',
                 'Княжна Марья возвратилась в свою комнату с грустным, испуганным выражением, которое редко покидало ее и делало ее некрасивое, болезненное лицо еще более некрасивым, села за свой письменный стол, уставленный миниатюрными портретами и заваленный тетрадями и книгами. Княжна была столь же беспорядочна, как отец ее порядочен. Она положила тетрадь геометрии и нетерпеливо распечатала письмо.']

In [66]:
RU_ABC = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

## 1. Базовый частотный метод по Шерлоку Холмсу
Задачи:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [200]:
def process_text(text: str) -> str:
  text = text.lower()
  text = ''.join([char for char in text if char in RU_ABC])
  words = nltk.word_tokenize(text)
  return ' '.join(words)

def encrypt_text(text: str, key: list) -> str:
  alph = list(RU_ABC)
  return ''.join([key[alph.index(char)] for char in text])

def decrypt_text(text: str, corpus_freq: Counter) -> str:
  text_freq = Counter(text)
  text_freq = [v[0] for v in text_freq.most_common()]
  corpus_freq = [v[0] for v in corpus_freq.most_common()]
  text_freq.extend(list(set(corpus_freq) - set(text_freq)))

  return ''.join([corpus_freq[text_freq.index(char)] for char in text])

def get_accuracy(original_text: str, decoded_text: str) -> float:
  text_len = len(original_text)
  assert text_len == len(decoded_text), 'length of texts must be the same'
  overlaps = [1 for i in range(text_len) if original_text[i] == decoded_text[i]]
  return sum(overlaps) / text_len

In [59]:
text_ru_cleaned = process_text(text_ru)
print(text_ru_cleaned[:200])

война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философииэта книга из разряда вечных потому что она обо всем о 


In [60]:
# Частотность букв по корпусам
corpus_freq = Counter(text_ru_cleaned)
print(corpus_freq)

Counter({' ': 385114, 'о': 223691, 'е': 166169, 'а': 162313, 'н': 133258, 'и': 129712, 'т': 115258, 'с': 103252, 'л': 98191, 'в': 91386, 'р': 80859, 'к': 67788, 'д': 58019, 'м': 56467, 'у': 53583, 'п': 47940, 'я': 42921, 'ь': 38351, 'г': 36870, 'ы': 36447, 'б': 34028, 'з': 32723, 'ч': 31207, 'ж': 21480, 'й': 21072, 'ш': 17158, 'х': 15586, 'ю': 12307, 'э': 6648, 'ц': 6172, 'щ': 5568, 'ф': 2990, 'ъ': 695, 'ё': 462})


In [62]:
example_text_1 = process_text(example_texts[0])
example_text_2 = process_text(example_texts[1])
example_text_3 = process_text(example_texts[2])

In [84]:
# Делаем ключ для шифрования текста
random.seed(23)
key = list(RU_ABC)
shuffle(key)

In [85]:
# Зашифрованные примеры текстов
ciphertext_1 = encrypt_text(example_text_1)
ciphertext_2 = encrypt_text(example_text_2)
ciphertext_3 = encrypt_text(example_text_3)

In [144]:
# Расшифрованные тексты
decrypt_text_1 = decrypt_text(ciphertext_1, corpus_freq)
decrypt_text_2 = decrypt_text(ciphertext_2, corpus_freq)
decrypt_text_3 = decrypt_text(ciphertext_3, corpus_freq)

In [181]:
print(f'Расшифрованный текст №1: \n{decrypt_text_1} \nсовпадение символов: {round(get_accuracy(example_text_1, decrypt_text_1), 2)}')
print(f'Расшифрованный текст №2: \n{decrypt_text_2} \nсовпадение символов: {round(get_accuracy(example_text_2, decrypt_text_2), 2)}')
print(f'Расшифрованный текст №3: \n{decrypt_text_3} \nсовпадение символов: {round(get_accuracy(example_text_3, decrypt_text_3), 2)}')

Расшифрованный текст №1: 
н то нвигп уеу м вортондч тесюинела н яели йиртох есьлия ыок янмуа от мртелорта шелжйананйач гмядуестон а мртелди ошаюаестд а ыоневе ьотонала мзас р ьвешог биямчондг ркилелрп йиртох мзи мкев коутове обэпнала цто секизкд у ндяковонлисащ сит болжсогм кесе бдле ьлмчеп арыоникж а ывацертаи килела ываьотонлисап клп робовонесап а н коги бдле рмитсп а твиноье озакесап обдусониссди н теуаи гасмтд нси коге яе новотега толыаларж рувднепрж от ыокэиязещфач ъуаыезих ьвобонфауа озакеп боьетоьо яеуеяе се ыочовосд ьвеше 
совпадение символов: 0.35
Расшифрованный текст №2: 
пмад даыитыинм те тнар петилетосмнч нтеаш дукетеьисавмлиха лопдотивнч к ьитолбику кеседяш ело ару укогово кок севмке олло рижошветло нкдявонм ел горасив бсе тгйвчья тнаж зятыиж т керлоса зевмыа бар н вюзепяснстер и н убонсиар унсдаривинм ло лайе ел горасив бсе тна падаыапсятовинм укогяточ ло лайе йвогори кок зуьсе не нсдожер и ьоэа н пеьезенсдонсиар 
совпадение символов: 0.29
Расшифрованный текст №3: 
княгне

**Вывод:** Как видно по качеству текста и по полученному скору - данный метод не эффективный.

## 2. Пункт 1 с биграмамми
Задачи:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм

In [209]:
def decrypt_text_with_ngrams(text: str, n_gram: int, origin_ngram_freq: Counter) -> str:
  text_n_gram_freq = Counter(ngrams(list(text), n_gram))
  text_n_gram_freq = [v[0] for v in text_n_gram_freq.most_common()]
  origin_ngram_freq = [v[0] for v in origin_ngram_freq.most_common()]
  text_n_gram_freq.extend(list(set(origin_ngram_freq) - set(text_n_gram_freq)))
  end = ''.join(origin_ngram_freq[text_n_gram_freq.index(tuple(text[-2:]))])[1]
  decrypted_text = ''.join([origin_ngram_freq[text_n_gram_freq.index(tuple(text[i:i+2]))][0] 
                  for i in range(len(text)) if i < len(text) - 1]) + end
  return decrypted_text

In [152]:
# Частотность биграм по корпусам и примера текста с лучшим скором из пункта 1
n_gram = 2
text_ngram_freq = Counter(ngrams(list(text_ru_cleaned), n_gram))
example_text_ngram_freq = Counter(ngrams(list(example_text_3), n_gram))

In [153]:
print(f'Количество биграм в оригинальных корпусах: {len(text_ngram_freq)}')
print(f'Количество биграм в оригинальных корпусах: {len(example_text_ngram_freq)}')

Количество биграм в оригинальных корпусах: 854
Количество биграм в оригинальных корпусах: 170


In [210]:
decrypt_text = decrypt_text_with_ngrams(ciphertext_3, n_gram, text_ngram_freq)

In [211]:
# Полученный результат
decrypt_text

'влвр озсиети ядаееглн пру ю тя о осв гчм ота  кс окнпонсосс ок лнема  хмк окл  тароикоьт эиммооьнланитте ооьнлаейитепдояталанранй сатаресвьныеаланрлаейитепдолок ае ооыо тяе твправ с в  клнси к дор с в заи  гдшх оаит  чеон  аиниоыдпор с в амнеечеааини в но аи влвр ол б о клнбуеаалзоо    бжс о идлекмиинлат    бжс сед от ношун оамнеечиутиусвнеодиниейнмег ееояьрепо отг о отвпракр'

In [212]:
print(f'Совпадение букв при использовании биграмм: {round(get_accuracy(example_text_3, decrypt_text), 2)}')

Совпадение букв при использовании биграмм: 0.07


**Вывод:** данный метод показывает результат хуже чем частотный метод по символам, что не удивительно — большая разница между количеством биграмм полученных из корпусов и примера текста, из за чего:
1. Проблематично сопоставить частотное распределение биграмм, так как в коротком тексте их слишком мало.
2. Появляется больше ошибок из за несоответсвия частотного распредления биграмм.

## 3. Метод обучения на основе MCMC-сэмплировании
Задачи:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.

**Комментарий:**
Можем рассмотреть текст поделенный на биграммы как Марковскую цепь, где вероятностями перехода по цепи и будут частоты биграмм.

Для обучения перестановки ключа шифрования будем считать вероятность пораждения заданного текста как произведение вероятностей биграмм заданного текста.

Так как количество возможных перестановок порядка 10 в 34 степени, то для оптимизации воспользуемся MCMC-сэмплированием.

Алгоритм действий:
1. Выбираем рандомный ключ шифрования.
2. Расшифровываем текст с текущим ключом и новым ключем с одной перестановкой.
3. Используя скоринговую функцию высчитываем скоры для результатов с использованием текущего и нового ключа.
4. Если скор с новым ключом лучше, то делаем его текущим.
5. В противном случае "подбрасываем монетку" (число от 0 до 1) и если оно меньше отношения скора с новым ключом к скору с текущим ключом, то делаем новый ключ текущим.
6. Повторяем с шага 2.

## 4. Расшифровка сообщения
Задача — расшифровать сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏